In [41]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import mglearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

# Data Preparation and Feature Engineering:

In [42]:
data = pd.read_csv("Energi_Viborg_Dandas_data.csv")

In [43]:
#drop columns not needed after asking the company about the meaning of these features

columns_to_be_removed = ['ID', 'mslink', 'XKoordinat','YKoordinat','LedningID','Dobbeltled','EjerKompon','SystemKode','KategoriAf','DatoUdf']
data=data.drop(columns_to_be_removed,axis='columns')

In [44]:
# in the column DatoSaneri is the date of repairing and if there is no date it means it is not repaired

data['DatoSaneri'].fillna(0, inplace=True)

In [45]:
# take only the pipes that are broken(by TV insection) now and the repaired ones

data_with_TVObsAndSaneri = data[data['TVObsKode'].isin([1]) | data['DatoSaneri'] > 0]

In [46]:
#get around 2077 rows with not broken pipes

data_not_broken = data[~data['TVObsKode'].isin([0]) | data['DatoSaneri'] == 0]
data_not_broken = data_not_broken.sample(n=2077) 

In [47]:
frames = [data_with_TVObsAndSaneri, data_not_broken]
  
data_final = pd.concat(frames)
data_final

,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
36,34.72,33.48,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,2010,2014,1997.0
42,39.46,39.16,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,2010,2014,0.0
43,39.71,39.48,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,2010,2014,0.0
64,40.55,40.08,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0
65,40.38,40.55,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4181,41.50,40.90,60.09,9.985022,200.0,1.0,1947.0,1,0,0.0,2010,2014,0.0
22449,7.10,6.79,31.49,9.844395,150.0,1.0,1982.0,1,0,0.0,2017,2017,0.0
2521,48.03,47.32,36.95,19.215156,200.0,1.0,1950.0,1,0,0.0,2010,2014,0.0
3862,32.58,31.45,71.78,15.742547,250.0,1.0,1991.0,1,0,0.0,2010,2014,0.0


In [48]:
data = data_final

In [49]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [50]:
#get data copied
datacopy = data


# add  age column

#get current year
from datetime import date
now = date.today().year


def age_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] > 0) :
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])

datacopy['Age'] = datacopy.apply(age_df, axis = 1)

In [51]:
# add a column 'PipeStatus'
# 1 as broken and 0 as not broken

def broken_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] < (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 1
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] >= (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 0
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return 1
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return 0
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return 0

datacopy['PipeStatus'] = datacopy.apply(broken_df, axis = 1)

In [52]:
# datacopy = datacopy.sample(n=22) 
# datacopy

In [53]:
# data_fs= np.where(np.isnan(datacopy))
# data_fs
# row = datacopy.iloc[369] #index=1 => second row
# print(row)

In [54]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(datacopy.shape[0]))
datacopy = datacopy.dropna()
print("Number of rows after removing NaNs: {}".format(datacopy.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [55]:
#drop columns not needed after adding new features

columns_to_be_removed = ['DatoOprett', 'DatoOpdate']
datacopy=datacopy.drop(columns_to_be_removed,axis='columns')
datacopy[0:-1]

,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoSaneri,Age,PipeStatus
36,34.72,33.48,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,1997.0,24.0,0
42,39.46,39.16,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,0.0,82.0,1
43,39.71,39.48,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,0.0,82.0,1
64,40.55,40.08,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1
65,40.38,40.55,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7867,35.61,35.50,24.16,4.552980,500.0,1.0,1985.0,1,0,0.0,0.0,36.0,0
4181,41.50,40.90,60.09,9.985022,200.0,1.0,1947.0,1,0,0.0,0.0,74.0,0
22449,7.10,6.79,31.49,9.844395,150.0,1.0,1982.0,1,0,0.0,0.0,39.0,0
2521,48.03,47.32,36.95,19.215156,200.0,1.0,1950.0,1,0,0.0,0.0,71.0,0


In [56]:
# creating features set and target

columns_to_be_removed = ['Age']
data_features= datacopy.drop(columns_to_be_removed,axis='columns')
columns_to_be_removed = ['fra_kote','til_kote', 'Laengde','Fald','DiameterIn','MaterialeK','anlag_aar','TransportK',
                         'Funktionsk','TVObsKode','DatoSaneri','PipeStatus']
data_target=datacopy.drop(columns_to_be_removed,axis='columns')

In [57]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


# Tuning Alpha for Lasso Model with Train-Test split and Normalization:

In [58]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(data_features, data_target, random_state=42)

In [60]:
# Learn the model with a certain numnber of alphas
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)
print("Best alpha value found: {}".format(lassocv.alpha_))

Best alpha value found: 0.00027171941816504954


In [61]:
# coefficients associated with the chosen alpha
pd.Series(lassocv.coef_ , index = data_features.columns)

fra_kote       0.000000
til_kote       0.004909
Laengde        0.008660
Fald          -0.005480
DiameterIn    -0.001933
MaterialeK     0.057187
anlag_aar     -0.739042
TransportK     0.000000
Funktionsk     0.022412
TVObsKode     -9.599596
DatoSaneri    -0.020684
PipeStatus    12.913336
dtype: float64

In [62]:
# to chech model performance
lasso = Lasso(max_iter = 10000, normalize = True)
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
mse = mean_squared_error(y_test, lasso.predict(X_test))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 90.00606557219152


The $R^2$ corresponding to this value for alpha are:

In [63]:
# R^2 of the associated alpha
print("R^2 on train data is {} and on test data is {}".format(lasso.score(X_train, y_train), 
                                                              lasso.score(X_test,y_test)))

R^2 on train data is 0.8330479082635659 and on test data is 0.8330339382074528


# Tuning Alpha for Lasso Model with validation set split and Normalization:

In [64]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=43)

In [65]:
best_score = 100
for alphas in 10**np.linspace(-10, 10, 100):
    # Learn the model with a certain numnber of alphas
    lasso = Lasso(max_iter = 10000, normalize=True, alpha=alphas)
    lasso.fit(X_train, y_train)
    
    # Evaluate the model
    score = mean_squared_error(y_val, lasso.predict(X_val))
    
    
    # If improvement, store score and parameter
    if score < best_score:
        best_score = score
        best_alphas = alphas

# Build a model on the combine training and valiation data
lasso = Lasso(max_iter = 10000, normalize=True, alpha = best_alphas)
lasso.fit(X_trainval, y_trainval)

print("Best alpha found: {}".format(best_alphas))
print("Best MSE on validation set: {}".format(best_score))
print("MSE on training/validation set: {}".format(mean_squared_error(y_trainval, lasso.predict(X_trainval))))
print("MSE on test set: {}".format(mean_squared_error(y_test, lasso.predict(X_test))))

Best alpha found: 1e-10
Best MSE on validation set: 89.95939707096159
MSE on training/validation set: 90.6420980883334
MSE on test set: 90.12324719782198


In [66]:
# to chech model performance
lasso1 = Lasso(max_iter = 10000, alpha = best_alphas, normalize = True)
lasso1.fit(X_train, y_train)
mse = mean_squared_error(y_test, lasso1.predict(X_test))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 89.32316048512754


In [67]:
# coefficients associated with the chosen alpha
pd.Series(lasso.coef_ , index = data_features.columns)

fra_kote      -0.032808
til_kote       0.039080
Laengde        0.010028
Fald          -0.004809
DiameterIn    -0.002068
MaterialeK     0.060376
anlag_aar     -0.740557
TransportK     0.000000
Funktionsk     0.038120
TVObsKode    -10.763722
DatoSaneri    -0.020689
PipeStatus    14.077606
dtype: float64

In [68]:
# R^2 of the associated alpha
print("R^2 on train data is {} and on test data is {}".format(lasso1.score(X_train, y_train), 
                                                              lasso1.score(X_test,y_test)))

R^2 on train data is 0.8334858109709405 and on test data is 0.8343007636401645


# Tuning Alpha for Lasso Model with cross validation split and Normalization:

In [69]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)

In [70]:
best_score = 0
for alphas in 10**np.linspace(-10, 10, 100):
    # Set a certain number of alphas
    lasso1 = Lasso(max_iter = 10000, normalize=True, alpha=alphas)
    
    # Perform cross validation
    scores = cross_val_score(lasso1, X_trainval, y_trainval, cv=5)
    
    # Compute the mean score
    score = scores.mean()
    
    # If improvement, store score and parameter
    if score > best_score:
        best_score = score
        best_alphas = alphas

# Build a model on the combine training and valiation data
lasso1 = Lasso(max_iter = 10000, normalize=True, alpha = best_alphas)
lasso1.fit(X_trainval, y_trainval)

print("Best alpha found: {}".format(best_alphas))
print("Best MSE on validation set: {}".format(best_score))
print("MSE on training/validation set: {}".format(mean_squared_error(y_trainval, lasso1.predict(X_trainval))))
print("MSE on test set: {}".format(mean_squared_error(y_test, lasso1.predict(X_test))))

Best alpha found: 0.0001830738280295366
Best MSE on validation set: 0.8304830920697327
MSE on training/validation set: 90.65274115883342
MSE on test set: 90.04350623318487


In [71]:
# to chech model performance
lasso = Lasso(max_iter = 10000, alpha = best_alphas, normalize = True)
lasso.fit(X_train, y_train)
mse = mean_squared_error(y_test, lasso.predict(X_test))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 89.26292377587706


In [72]:
# coefficients associated with the chosen alpha
pd.Series(lasso.coef_ , index = data_features.columns)

fra_kote       0.000000
til_kote       0.007438
Laengde        0.010585
Fald          -0.005505
DiameterIn    -0.002681
MaterialeK     0.088369
anlag_aar     -0.737195
TransportK     0.000000
Funktionsk    -0.019330
TVObsKode     -8.563289
DatoSaneri    -0.020750
PipeStatus    12.021491
dtype: float64

In [73]:
# R^2 of the associated alpha
print("R^2 on train data is {} and on test data is {}".format(lasso.score(X_train, y_train), 
                                                              lasso.score(X_test,y_test)))

R^2 on train data is 0.8334675351480569 and on test data is 0.8344125059550291


# Tuning Alpha for Lasso Model with Train-Test split and Standardization:

In [96]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(data_features, data_target, random_state=42)

# preprocessing using 0-1 scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [97]:
# Learn the model with a certain numnber of alphas
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)
print("Best alpha value found: {}".format(lassocv.alpha_))

Best alpha value found: 0.00027171941816504954


In [98]:
# ridge.fit(data_features, data_target)
pd.Series(lassocv.coef_ , index = data_features.columns)

fra_kote       0.000000
til_kote       0.004909
Laengde        0.008660
Fald          -0.005480
DiameterIn    -0.001933
MaterialeK     0.057187
anlag_aar     -0.739042
TransportK     0.000000
Funktionsk     0.022412
TVObsKode     -9.599596
DatoSaneri    -0.020684
PipeStatus    12.913336
dtype: float64

In [99]:
# to chech model performance
lasso = Lasso(max_iter = 10000)
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
mse = mean_squared_error(y_test, lasso.predict(X_test))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 90.11525659991264


In [100]:
# R^2 of the associated alpha
print("R^2 on train data is {} and on test data is {}".format(lasso.score(X_train_scaled, y_train), 
                                                              lasso.score(X_test_scaled,y_test)))

R^2 on train data is -3992.421579915157 and on test data is -4027.534565341788


# Tuning Alpha for Ridge Model with validation set split and Standardization:

In [101]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=43)

# preprocessing using zero mean and unit variance scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)
X_trainval_scaled = scaler.transform( X_trainval)


In [102]:
best_score = 100
for alphas in 10**np.linspace(-10, 10, 100):
    # Learn the model with a certain numnber of alphas
    lasso1 = Lasso(max_iter = 10000, alpha=alphas)
    lasso1.fit(X_train_scaled, y_train)
    
    # Evaluate the model
    score = mean_squared_error(y_val, lasso1.predict(X_val_scaled))
    
    
    # If improvement, store score and parameter
    if score < best_score:
        best_score = score
        best_alphas = alphas

# Build a model on the combine training and valiation data
lasso1 = Lasso(max_iter = 10000, alpha = best_alphas)
lasso1.fit(X_trainval_scaled, y_trainval)

print("Best alpha found: {}".format(best_alphas))
print("Best MSE on validation set: {}".format(best_score))
print("MSE on training/validation set: {}".format(mean_squared_error(y_trainval, lasso1.predict(X_trainval_scaled))))
print("MSE on test set: {}".format(mean_squared_error(y_test, lasso1.predict(X_test_scaled))))

Best alpha found: 1e-10
Best MSE on validation set: 89.959397039184
MSE on training/validation set: 90.6420980883334
MSE on test set: 90.12324722865576


In [103]:
# to chech model performance
lasso = Lasso(max_iter = 10000, alpha = best_alphas, normalize = True)
lasso.fit(X_train_scaled, y_train)
mse = mean_squared_error(y_test, lasso.predict(X_test_scaled))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 89.3231604851273


In [104]:
# coefficients associated with the chosen alpha
pd.Series(lasso.coef_ , index = data_features.columns)

fra_kote      -0.494166
til_kote       0.614310
Laengde        0.326611
Fald          -0.304150
DiameterIn    -0.545949
MaterialeK     0.488447
anlag_aar    -15.121566
TransportK     0.000000
Funktionsk    -0.035154
TVObsKode     -3.972688
DatoSaneri   -18.974197
PipeStatus     5.356566
dtype: float64

In [105]:
# R^2 of the associated alpha
print("R^2 on train data is {} and on test data is {}".format(lasso.score(X_train_scaled, y_train), 
                                                              lasso.score(X_test_scaled,y_test)))

R^2 on train data is 0.8334858109709408 and on test data is 0.8343007636401649


# Tuning Alpha for Ridge Model with cross validation split and Standardization:

In [106]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)

# preprocessing using 0-1 scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_test_scaled = scaler.transform(X_test)
X_trainval_scaled = scaler.transform( X_trainval)

In [107]:
best_score = 0
for alphas in 10**np.linspace(-10, 10, 100):
    # Set a certain number of alphas
    lasso1 = Lasso(max_iter = 10000, alpha=alphas)
    
    # Perform cross validation
    scores = cross_val_score(lasso1, X_trainval_scaled, y_trainval, cv=5)
    
    # Compute the mean score
    score = scores.mean()
    
    # If improvement, store score and parameter
    if score > best_score:
        best_score = score
        best_alphas = alphas

# Build a model on the combine training and valiation data
lasso1 = Lasso(max_iter = 10000, alpha = best_alphas)
lasso1.fit(X_trainval_scaled, y_trainval)

print("Best alpha found: {}".format(best_alphas))
print("Best MSE on validation set: {}".format(best_score))
print("MSE on training/validation set: {}".format(mean_squared_error(y_trainval, lasso1.predict(X_trainval_scaled))))
print("MSE on test set: {}".format(mean_squared_error(y_test, lasso1.predict(X_test_scaled))))

Best alpha found: 0.007564633275546291
Best MSE on validation set: 0.8304826641804549
MSE on training/validation set: 90.64924247884153
MSE on test set: 90.06496944050436


In [108]:
# to chech model performance
lasso = Lasso(max_iter = 10000, alpha = best_alphas)
lasso.fit(X_trainval_scaled,y_trainval)
mse = mean_squared_error(y_test, lasso.predict(X_test_scaled))
print("The MSE associated with alpha value: {}".format(mse))

The MSE associated with alpha value: 90.06496944050436


In [109]:
# coefficients associated with the chosen alpha
pd.Series(lasso.coef_ , index = data_features.columns)

fra_kote       0.000000
til_kote       0.084861
Laengde        0.252207
Fald          -0.365854
DiameterIn    -0.396374
MaterialeK     0.317354
anlag_aar    -15.155918
TransportK     0.000000
Funktionsk     0.048928
TVObsKode     -4.294270
DatoSaneri   -18.913032
PipeStatus     5.613163
dtype: float64

In [110]:
# R^2 of the associated alpha
print("R^2 on train/validation data is {} and on test data is {}".format(lasso.score(X_trainval_scaled,y_trainval), 
                                                              lasso.score(X_test_scaled,y_test)))

R^2 on train/validation data is 0.8330716593307329 and on test data is 0.8329246683837579
